In [16]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.externals import joblib
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV
from sklearn.decomposition import TruncatedSVD

In [2]:
labels = pd.read_pickle('labels_df')

In [3]:
sample = labels.ix[0:10]

In [4]:
explode_probabilities = lambda x: pd.Series({j: k['probability'] for j, k in x.items()})

labels = pd.concat([labels, labels.countries.apply(explode_probabilities)], axis=1)

In [5]:
non_country_columns = ['raw_text_url', 'title', 'toc_subject', 'topics', 'raw_text', 'entities', 'countries']
country_columns = sorted(list(set(labels.columns.tolist()) - set(non_country_columns)))

In [6]:
labels

,raw_text_url,title,toc_subject,topics,raw_text,entities,countries,Afghanistan,Albania,Algeria,...,Uruguay,Uzbekistan,Vanuatu,"Venezuela, Bolivarian Republic of",Viet Nam,"Virgin Islands, British","Virgin Islands, U.S.",Yemen,Zambia,Zimbabwe
0,https://www.federalregister.gov/articles/text/...,Culturally Significant Objects Imported for Ex...,Culturally Significant Objects Imported for Ex...,[],\nSUMMARY: \nNotice is hereby given of the fol...,"[(Mauritshuis, ORGANIZATION), (United States, ...","{u'Canada': {u'count': 1, u'probability': 0.07...",NaN,NaN,NaN,...,0.071429,NaN,NaN,0.071429,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.federalregister.gov/articles/text/...,Proposed Collection: Comment Request,None,[],\nACTION: \nNotice and request for comments. \...,"[(Treasury, ORGANIZATION), (Treasury, ORGANIZA...","{u'United States': {u'count': 1, u'probability...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.federalregister.gov/articles/text/...,National Medal of Technology and Innovation Ca...,Calls for Nominations:,[],\nACTION: \nNotice and request for nominations...,"[(Department of Commerce, ORGANIZATION), (Unit...","{u'Canada': {u'count': 1, u'probability': 0.08...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.federalregister.gov/articles/text/...,Additional Designations of Individuals Pursuan...,Blocking or Unblocking of Persons and Property:,[],\nACTION: \nNotice. \nSUMMARY: \nThe U.S. Depa...,"[(U.S. Department of the Treasury, ORGANIZATIO...","{u'United Kingdom': {u'count': 1, u'probabilit...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.federalregister.gov/articles/text/...,Fisheries of the Northeastern United States; S...,Fisheries of the Northeastern United States:,[],\nACTION: \nTemporary rule; quota transfer. \n...,"[(NMFS, ORGANIZATION), (North Carolina, LOCATI...","{u'United States': {u'count': 7, u'probability...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,https://www.federalregister.gov/articles/text/...,Announcing an Open Meeting of the Information ...,Meetings:,[],\nACTION: \nNotice. \nSUMMARY: \nThe Informati...,"[(Privacy Advisory Board, ORGANIZATION), (Unit...","{u'United States': {u'count': 3, u'probability...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,https://www.federalregister.gov/articles/text/...,Polyethylene Retail Carrier Bags From the Peop...,Antidumping Duty Administrative Reviews; Resul...,[],\nSUMMARY: \nThe Department of Commerce (the D...,"[(Department of Commerce, ORGANIZATION), (Chin...","{u'United States': {u'count': 3, u'probability...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,https://www.federalregister.gov/articles/text/...,Proposed Collections; Comment Requests,None,[],\nACTION: \nNotice and request for comments. \...,"[(Treasury, ORGANIZATION), (Office of Manageme...","{u'Brazil': {u'count': 1, u'probability': 0.5}...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,https://www.federalregister.gov/articles/text/...,In the Matter of: Emenike Charles Nwankwoala C...,Orders Denying Export Privileges:,[],\nOrder Denying Export Privileges \nOn January...,"[(U.S. District Court, ORGANIZATION), (Marylan...","{u'United States': {u'count': 4, u'probability...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,https://www.federalregister.gov/articles/text/...,Fisheries Off West Coast States; Pacific Coast...,Fisheries off West Coast States:,"[Fisheries, Fishing]",\nACTION: \nProposed rule; request for comment...,[(Pacific Coast Groundfish Fishery Trawl Ratio...,"{u'Brazil': {u'count': 1, u'probability': 0.66...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
stopped = stopwords.words('english')

In [8]:
# maybe should make the zero into a nan again so that the model doesnt think that it is zero?
# or maybe zero is correct because the chance of that country is zero?
# but then the model will always predict zero for documents that it is trained on

labels[['title', 'toc_subject']] = labels[['title', 'toc_subject']].fillna('')
documents = labels.title + ' ' + labels.toc_subject + ' ' + labels.topics.apply(lambda x: ' '.join(x)) + labels.raw_text

# vec = TfidfVectorizer(sublinear_tf=True, ngram_range=(1,3), stop_words=stopped)
vec = TfidfVectorizer(sublinear_tf=True, ngram_range=(1,3), stop_words=stopped, max_features=500000)
#tfidf vectorizer automatically normalizes
# tfidf = vec.fit_transform(documents.tolist())
# joblib.dump(tfidf, 'tfidf.pkl')
tfidf = joblib.load('tfidf.pkl')

In [17]:
svd = TruncatedSVD(n_components=100, random_state=42)
# lsa = svd.fit_transform(tfidf)
# joblib.dump(lsa, 'lsa.pkl')
lsa = joblib.load('lsa.pkl')

In [18]:
def probability_model(country):
    # filling in the blanks with a predictive model
#     X = tfidf
    X = lsa # lose some accuracy to speed things up
    est = LinearSVC(penalty='l1', dual=False)
    est = CalibratedClassifierCV(est, cv=3) # increase cv for better accuracy
    y = labels[country].apply(lambda x: 1 if x>0.5 else 0)
    try:
        est.fit(X, y)
        prediction = est.predict(X)
        t = labels[[country]]
        t['labeled_class'] = y
        t['predicted_class'] = prediction
        t['probability'] = [i[1] for i in est.predict_proba(X)]
    #     temp = t[(t['predicted_class'] == 1) & (t['labeled_class']== 0)]
    #     print classification_report(y, prediction)
    #     print temp.shape
    #     print temp
        return t.probability
    except Exception as e:
        print(country, e)
        return y

In [19]:
for country in country_columns:
    t = probability_model(country)

    # keep the original probability where greater than the modeled probability
    labels[country] = labels[country].where(labels[country] > t, t)
    
    # returns errors where not enough pre-labeled data for that country

/Users/amangum/anaconda/envs/datasci/lib/python2.7/site-packages/IPython/kernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/amangum/anaconda/envs/datasci/lib/python2.7/site-packages/IPython/kernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/amangum/anaconda/envs/datasci/lib/python2.7/site-packages/IPython/kernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the c

(u'Andorra', ValueError('This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0',))
(u'Anguilla', ValueError('This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0',))
(u'Bhutan', ValueError('Requesting 3-fold cross-validation but provided less than 3 examples for at least one class.',))
(u'Bolivia, Plurinational State of', ValueError('This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0',))
(u'Bonaire, Sint Eustatius and Saba', ValueError('This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0',))
(u'Bosnia and Herzegovina', ValueError('Requesting 3-fold cross-validation but provided less than 3 examples for at least one class.',))
(u'Botswana', ValueError('Requesting 3-fold cross-validation but provided less than 3 examples for at least one class.',))
(u'Comoros', ValueError('Requesting 3-fold cross-

In [24]:
labels = labels.rename(columns={'countries':'pre-labeled countries'})

In [26]:
labels

,raw_text_url,title,toc_subject,topics,raw_text,entities,pre-labeled countries,Afghanistan,Albania,Algeria,...,Uruguay,Uzbekistan,Vanuatu,"Venezuela, Bolivarian Republic of",Viet Nam,"Virgin Islands, British","Virgin Islands, U.S.",Yemen,Zambia,Zimbabwe
0,https://www.federalregister.gov/articles/text/...,Culturally Significant Objects Imported for Ex...,Culturally Significant Objects Imported for Ex...,[],\nSUMMARY: \nNotice is hereby given of the fol...,"[(Mauritshuis, ORGANIZATION), (United States, ...","{u'Canada': {u'count': 1, u'probability': 0.07...",0.002095,0.000932,0.001163,...,0.071429,0.000988,0.000697,0.071429,0.007190,0.000977,0.005605,0.001408,0.000666,0.001488
1,https://www.federalregister.gov/articles/text/...,Proposed Collection: Comment Request,,[],\nACTION: \nNotice and request for comments. \...,"[(Treasury, ORGANIZATION), (Treasury, ORGANIZA...","{u'United States': {u'count': 1, u'probability...",0.002207,0.001381,0.001117,...,0.000079,0.000997,0.000319,0.003269,0.009604,0.000657,0.004183,0.001346,0.000666,0.001565
2,https://www.federalregister.gov/articles/text/...,National Medal of Technology and Innovation Ca...,Calls for Nominations:,[],\nACTION: \nNotice and request for nominations...,"[(Department of Commerce, ORGANIZATION), (Unit...","{u'Canada': {u'count': 1, u'probability': 0.08...",0.002271,0.000751,0.001401,...,0.003621,0.002398,0.000784,0.003825,0.007944,0.001174,0.004918,0.001978,0.000666,0.001691
3,https://www.federalregister.gov/articles/text/...,Additional Designations of Individuals Pursuan...,Blocking or Unblocking of Persons and Property:,[],\nACTION: \nNotice. \nSUMMARY: \nThe U.S. Depa...,"[(U.S. Department of the Treasury, ORGANIZATIO...","{u'United Kingdom': {u'count': 1, u'probabilit...",0.008639,0.000797,0.001883,...,0.003442,0.005527,0.000299,0.008596,0.008004,0.003873,0.004249,0.002994,0.000666,0.007600
4,https://www.federalregister.gov/articles/text/...,Fisheries of the Northeastern United States; S...,Fisheries of the Northeastern United States:,[],\nACTION: \nTemporary rule; quota transfer. \n...,"[(NMFS, ORGANIZATION), (North Carolina, LOCATI...","{u'United States': {u'count': 7, u'probability...",0.001133,0.000193,0.001305,...,0.001923,0.001425,0.001711,0.002086,0.009294,0.001430,0.007493,0.002212,0.000666,0.001492
5,https://www.federalregister.gov/articles/text/...,Announcing an Open Meeting of the Information ...,Meetings:,[],\nACTION: \nNotice. \nSUMMARY: \nThe Informati...,"[(Privacy Advisory Board, ORGANIZATION), (Unit...","{u'United States': {u'count': 3, u'probability...",0.001508,0.000384,0.001291,...,0.003303,0.001306,0.000657,0.003523,0.007204,0.001367,0.004065,0.002010,0.000666,0.001455
6,https://www.federalregister.gov/articles/text/...,Polyethylene Retail Carrier Bags From the Peop...,Antidumping Duty Administrative Reviews; Resul...,[],\nSUMMARY: \nThe Department of Commerce (the D...,"[(Department of Commerce, ORGANIZATION), (Chin...","{u'United States': {u'count': 3, u'probability...",0.002183,0.001123,0.001342,...,0.003307,0.001086,0.000084,0.003194,0.021544,0.000899,0.005416,0.001684,0.000666,0.001567
7,https://www.federalregister.gov/articles/text/...,Proposed Collections; Comment Requests,,[],\nACTION: \nNotice and request for comments. \...,"[(Treasury, ORGANIZATION), (Office of Manageme...","{u'Brazil': {u'count': 1, u'probability': 0.5}...",0.002443,0.001112,0.001241,...,0.002447,0.001201,0.000607,0.005274,0.008538,0.001086,0.005895,0.001657,0.000666,0.001608
8,https://www.federalregister.gov/articles/text/...,In the Matter of: Emenike Charles Nwankwoala C...,Orders Denying Export Privileges:,[],\nOrder Denying Export Privileges \nOn January...,"[(U.S. District Court, ORGANIZATION), (Marylan...","{u'United States': {u'count': 4, u'probability...",0.000637,0.000936,0.000782,...,0.003676,0.000722,0.000251,0.018658,0.011450,0.010297,0.003355,0.001640,0.000666,0.000916
9,https://www.federalregister.gov/articles/text/...,Fisheries Off West Coast States; Pac

In [27]:
labels.to_pickle('predicted_countries_df')